In [1]:
import scipy
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt

for m in [scipy, np, pd, matplotlib]:
    print(m.__name__, m.__version__)

from scipy.signal import savgol_filter
from scipy.signal import argrelextrema
from scipy.signal import find_peaks

from funcs.factor_utils import FactorUtil

scipy 1.15.2
numpy 1.26.4
pandas 2.2.3
matplotlib 3.10.1


In [2]:
bar_df_1 = FactorUtil.get_bar_df('FG2409_MIN_1', '20240813', '00:00:00', 300)
bar_df_1.head()

,O,H,L,C,V,AP,AV,BP,BV
ts,,,,,,,,,
2024-08-13 09:00:00,1278.0,1278.0,1276.0,1276.0,681,1278.000,218,1276.000,152
2024-08-13 09:01:00,1277.0,1282.0,1276.0,1278.0,5557,1279.737,21421,1278.345,17282
2024-08-13 09:02:00,1278.0,1279.0,1276.0,1278.0,1634,1278.102,43934,1277.119,31489
2024-08-13 09:03:00,1277.0,1282.0,1277.0,1281.0,3749,1280.911,33442,1279.997,33766
2024-08-13 09:04:00,1281.0,1281.0,1279.0,1280.0,1619,1280.149,41884,1279.158,46279


In [3]:
bar_df_3 = FactorUtil.get_window_bar_df('FG2409_MIN_1', '20240813')
bar_df_3.columns

20240812 22:51:00


Index(['O_9', 'H_9', 'L_9', 'C_9', 'V_9', 'AP_9', 'AV_9', 'BP_9', 'BV_9',
       'O_8', 'H_8', 'L_8', 'C_8', 'V_8', 'AP_8', 'AV_8', 'BP_8', 'BV_8',
       'O_7', 'H_7', 'L_7', 'C_7', 'V_7', 'AP_7', 'AV_7', 'BP_7', 'BV_7',
       'O_6', 'H_6', 'L_6', 'C_6', 'V_6', 'AP_6', 'AV_6', 'BP_6', 'BV_6',
       'O_5', 'H_5', 'L_5', 'C_5', 'V_5', 'AP_5', 'AV_5', 'BP_5', 'BV_5',
       'O_4', 'H_4', 'L_4', 'C_4', 'V_4', 'AP_4', 'AV_4', 'BP_4', 'BV_4',
       'O_3', 'H_3', 'L_3', 'C_3', 'V_3', 'AP_3', 'AV_3', 'BP_3', 'BV_3',
       'O_2', 'H_2', 'L_2', 'C_2', 'V_2', 'AP_2', 'AV_2', 'BP_2', 'BV_2',
       'O_1', 'H_1', 'L_1', 'C_1', 'V_1', 'AP_1', 'AV_1', 'BP_1', 'BV_1', 'O',
       'H', 'L', 'C', 'V', 'AP', 'AV', 'BP', 'BV'],
      dtype='object')

In [5]:
bar_df_3['R'] = (bar_df_3['C'] - bar_df_3['C_5']) / bar_df_3['C_5']
min_val, max_val = bar_df_3['R'].min(), bar_df_3['R'].max()
min_val, max_val

(-0.009060955518945634, 0.0113544201135442)

In [6]:
bins = pd.interval_range(start=min_val, end=max_val, periods=10)
bins

IntervalIndex([  (-0.009060955518945634, -0.00701941795569665],
                 (-0.00701941795569665, -0.004977880392447667],
               (-0.004977880392447667, -0.0029363428291986834],
                 (-0.0029363428291986834, -0.0008948052659497],
                  (-0.0008948052659497, 0.0011467322972992834],
                 (0.0011467322972992834, 0.003188269860548267],
                   (0.003188269860548267, 0.00522980742379725],
                   (0.00522980742379725, 0.007271344987046234],
                  (0.007271344987046234, 0.009312882550295219],
                    (0.009312882550295219, 0.0113544201135442]],
              dtype='interval[float64, right]')

In [9]:
bar_df_3['K1'] = pd.cut(bar_df_3['R'], bins=bins, labels=range(10), include_lowest=True)
bar_df_3['K1'].value_counts().sort_index()

K1
(-0.009060955518945634, -0.00701941795569665]        20
(-0.00701941795569665, -0.004977880392447667]        46
(-0.004977880392447667, -0.0029363428291986834]     217
(-0.0029363428291986834, -0.0008948052659497]       565
(-0.0008948052659497, 0.0011467322972992834]       1579
(0.0011467322972992834, 0.003188269860548267]       515
(0.003188269860548267, 0.00522980742379725]         161
(0.00522980742379725, 0.007271344987046234]          30
(0.007271344987046234, 0.009312882550295219]          5
(0.009312882550295219, 0.0113544201135442]            2
Name: count, dtype: int64

In [8]:
bar_df_3['K'] = pd.cut(bar_df_3['R'], bins=bins, labels=range(10), include_lowest=True).cat.codes
bar_df_3['K'].value_counts().sort_index()

K
-1       1
 0      20
 1      46
 2     217
 3     565
 4    1579
 5     515
 6     161
 7      30
 8       5
 9       2
Name: count, dtype: int64

In [10]:
bar_df_3['R1'] = (bar_df_3['C'] - bar_df_3['C_5'])
min_val_1, max_val_1 = bar_df_3['R1'].min(), bar_df_3['R1'].max()
min_val_1, max_val_1

(-11.0, 14.0)

In [11]:
bins_1 = pd.interval_range(start=min_val_1, end=max_val_1, periods=10)
bins_1

IntervalIndex([(-11.0, -8.5],  (-8.5, -6.0],  (-6.0, -3.5],  (-3.5, -1.0],
                 (-1.0, 1.5],    (1.5, 4.0],    (4.0, 6.5],    (6.5, 9.0],
                 (9.0, 11.5],  (11.5, 14.0]],
              dtype='interval[float64, right]')

In [14]:
# 分桶
# bar_df_3['K'] = pd.cut(bar_df_3['R'], bins=bins_1, labels=range(10), include_lowest=True).cat.codes
bar_df_3['K1'] = pd.cut(bar_df_3['R1'], bins=bins_1, labels=range(10), include_lowest=True)
# 值分布
bar_df_3['K1'].value_counts().sort_index()


K1
(-11.0, -8.5]      16
(-8.5, -6.0]       74
(-6.0, -3.5]      189
(-3.5, -1.0]     1115
(-1.0, 1.5]      1029
(1.5, 4.0]        594
(4.0, 6.5]         82
(6.5, 9.0]         32
(9.0, 11.5]         3
(11.5, 14.0]        2
Name: count, dtype: int64

In [15]:
bar_df_3['K0'] = pd.cut(bar_df_3['R1'], bins=bins_1, labels=range(10), include_lowest=True).cat.codes
bar_df_3['K0'].value_counts().sort_index()

K0
-1       5
 0      16
 1      74
 2     189
 3    1115
 4    1029
 5     594
 6      82
 7      32
 8       3
 9       2
Name: count, dtype: int64

In [1]:
import xgboost as xgb
# print(xgb.__version__)

XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/Users/yunhui/work/venv/ctp-py311/lib/python3.11/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib\n  Referenced from: <5F165A8A-F42A-39F1-8BD3-CC0F65C2BE0F> /Users/yunhui/work/venv/ctp-py311/lib/python3.11/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/local/opt/libomp/lib/libomp.dylib' (n